In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
#Nos conectamos al drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def crear_diccionario(dmel):
    grupo = dmel.groupby('Model_nodos')['Symbol'].first().reset_index()
    grupo = grupo.rename(columns={'Symbol': 'ID_nuevo'})
    dmel = dmel.merge(grupo, on='Model_nodos', how='left')
    diccionario = dmel[['Astyanax_ortologos', 'Symbol', 'ID_nuevo']].drop_duplicates()
    return diccionario

def actualizar_diccionario(nuevo_df, diccionario):
    grupo = nuevo_df.groupby('Model_nodos')['Symbol'].first().reset_index()
    grupo = grupo.rename(columns={'Symbol': 'ID_nuevo'})
    nuevo_df = nuevo_df.merge(grupo, on='Model_nodos', how='left')

    conocidos = diccionario[['Astyanax_ortologos', 'ID_nuevo']]
    df_match = nuevo_df.merge(conocidos, on='Astyanax_ortologos', how='inner', suffixes=('', '_previo'))
    df_match['ID_nuevo'] = df_match['ID_nuevo_previo']
    df_match = df_match.drop(columns=['ID_nuevo_previo'])

    df_no_match = nuevo_df[~nuevo_df['Astyanax_ortologos'].isin(conocidos['Astyanax_ortologos'])]
    df_final = pd.concat([df_match, df_no_match], ignore_index=True)

    nuevo_diccionario = df_final[['Astyanax_ortologos', 'Symbol', 'ID_nuevo']].drop_duplicates()
    diccionario_actualizado = pd.concat([diccionario, nuevo_diccionario], ignore_index=True)
    diccionario_actualizado = diccionario_actualizado.drop_duplicates(subset=['Astyanax_ortologos'])
    return diccionario_actualizado

In [4]:
dmel = pd.read_csv("/content/drive/My Drive/Sakura/Diccionarios/Dmel_Isoformas.csv")
homo = pd.read_csv("/content/drive/My Drive/Sakura/Diccionarios/Homo_Isoformas.csv")
mus  = pd.read_csv("/content/drive/My Drive/Sakura/Diccionarios/Mus_Isoformas.csv")

In [5]:
diccionario_df = crear_diccionario(dmel)

In [6]:
diccionario_df = actualizar_diccionario(homo, diccionario_df)

In [7]:
diccionario_df = actualizar_diccionario(mus, diccionario_df)

In [8]:
output_path = "/content/drive/My Drive/Sakura/Diccionarios/SF_diccionario.csv"
diccionario_df.to_csv(output_path, index=False)

In [9]:
diccionario_df.shape

(9196, 3)